# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo enriquecer os dados de clientes com dicas sobre o preço de fechamento dos ativos do banco.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar um serviço de cotações de ativos para informar o preço de fechamento ajustado dos ativos do banco. Essa mensagem deve enfatizar a oportunidade de investimento.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai entregar os dados enriquecidos em uma arquivo json que será consumido pelos serviços do banco Santander.



In [1]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [3]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
user_ids

[1, 2, 3, 4, 5]

In [5]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None


Recupere o preço de fechamento ajustado para as ações solicitadas do banco Santander e gere uma mensagem personalizada para os clientes.

In [14]:
import pandas_datareader.data as pdr
import yfinance
from datetime import date,timedelta

def cotacao_bolsa(ativos=[]):
  if len(ativos)<=0:
    return []

  yfinance.pdr_override()

  data_final = date.today()
  data_inicial = data_final - timedelta(7)

  tabela_cotacoes = pdr.get_data_yahoo(ativos, data_inicial, data_final)

  retorno = []

  if len(ativos) > 1:
    for ativo in ativos:
      retorno.append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": (f"Oportunidade de investimento!!, preço de fechamento atualizado do ativo {ativos[ativos.index(ativo)]} é R$ {round(tabela_cotacoes['Adj Close'][ativo].tail(1)[0],2)}")
      })

    return retorno
  else:
    retorno.append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": (f"Oportunidade de investimento!!, preço de fechamento atualizado do ativo {ativos[0]} é R$ {round(tabela_cotacoes['Adj Close'].tail(1)[0],2)}")
    })

    return retorno

## **T**ransform

Inclua no canal news de cada usuário informado as mensagens geradas sobre o preço de fechamento dos ativos.

In [15]:
users = [user for id in user_ids if (user := get_user(id)) is not None]

ativos = ['SANB11.SA','SANB4.SA','SANB3.SA',]

precos_fechamento = cotacao_bolsa(ativos)

for user in users:
  for item in precos_fechamento:
    user['news'].append(item)

print(json.dumps(users, indent=2))

[*********************100%%**********************]  3 of 3 completed
[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        "id": 5,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/i

## **L**oad

Salve um arquivo no formato json com os dados enriquecidos dos clientes para ser consumido pelos serviços do banco Santander.

In [19]:
load = pd.DataFrame(users)

load.to_json('SDW2023.json')